# Capstone Final Notebook

## Business Problem

A Buisness man wants to open up a chain of Indonesian Cusine restaurants in Mumbai. He would like to know the best places to open restaurants and also is thinking of getting some food trucks as well and how to position them.

## Data Set

The data set includes different parts of Mumbai which consists of diffirent Neighborhoods in them.


In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import folium
import requests
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json


from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

print('Libraries Imported')

Libraries Imported


In [2]:
data_url = 'https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Mumbai'
data_html = requests.get(data_url).text
soup = bs(data_html,'lxml')

In [3]:
import csv
csv_file = open('data_mumbai.csv', 'w', newline =  '')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Area','Neighborhood'])
areass = soup.find_all('h3')
areas = []
for a in areass:
    areas.append(a.text[:-6])
n = soup.find_all('li')
count = soup.find_all('ul')
count = count[5:39]
i = 0
for a in count:
    Area = areas[i]
    i+=1
    for b in a.find_all('li'):
        Neigh = b.text
        csv_writer.writerow([Area,Neigh])
csv_file.close()

In [4]:
Neighborhoods = pd.read_csv('data_mumbai.csv')
Neighborhoods.head()

,Area,Neighborhood
0,Andheri,Amboli
1,Andheri,Chakala
2,Andheri,D.N. Nagar
3,Andheri,Four Bungalows
4,Andheri,JB Nagar


### Now to get Longitude and Latitude of all the neighborhoods and add it to dataframe


In [5]:
Latitudes = []
Longitudes = []
for index, row in Neighborhoods.iterrows():
    address = '{}, {}'.format(row['Neighborhood'],'Mumbai')
    print(address)
    try:
        geolocator = Nominatim(user_agent="ny_explorer")
        location = geolocator.geocode(address)
        Latitudes.append(location.latitude)
        Longitudes.append(location.longitude)
    except Exception as e:
        print('not found')
        Latitudes.append(None)
        Longitudes.append(None)
        

Amboli, Mumbai
Chakala, Mumbai
D.N. Nagar, Mumbai
Four Bungalows, Mumbai
JB Nagar, Mumbai
not found
Lokhandwala, Mumbai
Marol, Mumbai
Model Town, Mumbai
Oshiwara, Mumbai
Poonam Nagar, Mumbai
Sahar, Mumbai
Saki Naka, Mumbai
Seven Bungalows, Mumbai
Versova, Mumbai
Mira Road, Mumbai
Bhayandar, Mumbai
Uttan, Mumbai
Kashimira, Mumbai
Bandra Kurla Complex, Mumbai
Bandstand Promenade, Mumbai
not found
Gandhi Nagar, Mumbai
Kherwadi, Mumbai
Land's End, Mumbai
Pali Hill, Mumbai
Old Town, Mumbai
Bandra Reclamation, Mumbai
I.C. Colony, Mumbai
L.I.C. Colony aka Jeevan Bhima Nagar, Mumbai
not found
Eksar Colony, Mumbai
not found
Shimpoli, Mumbai
Gorai, Mumbai
Kora Kendra, Mumbai
Vazira Naka, Mumbai
Babhai, Mumbai
Chikuwadi, Mumbai
not found
Yogi Nagar, Mumbai
Magathane, Mumbai
Nancy Colony, Mumbai
not found
Sukurwadi, Mumbai
Aarey Milk Colony, Mumbai
Motilal Nagar, Mumbai
Bangur Nagar, Mumbai
Gokuldham, Mumbai
Jayprakash Nagar, Mumbai
Pandurangwadi, Mumbai
not found
NESCO colony, Mumbai
not found
Jo

Adding Longitude and Latitude columns to Neighborhoods and removing the rows with no values.

In [6]:
Neighborhoods['Latitude'] = Latitudes
Neighborhoods['Longitude'] = Longitudes
Neighborhoods.dropna(axis = 0, inplace = True)
Neighborhoods

,Area,Neighborhood,Latitude,Longitude
0,Andheri,Amboli,19.127931,72.847735
1,Andheri,Chakala,19.115287,72.861808
2,Andheri,D.N. Nagar,19.128292,72.830193
3,Andheri,Four Bungalows,19.128794,72.825554
5,Andheri,Lokhandwala,19.143189,72.824081
6,Andheri,Marol,19.117220,72.882342
7,Andheri,Model Town,19.184102,72.943766
8,Andheri,Oshiwara,19.150244,72.834229
9,Andheri,Poonam Nagar,19.136918,72.869128
10,Andheri,Sahar,19.099595,72.867284


### Creating a map of mumbai and marking all the Neighboorhood

In [7]:
address = ' Mumbai, India'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai are 18.9387711, 72.8353355.


In [8]:
map_mumbai = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, area, neighborhood in zip(Neighborhoods['Latitude'], Neighborhoods['Longitude'], Neighborhoods['Area'], Neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, area)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mumbai)  
    
map_mumbai

### Using Foursquare api to explore

In [9]:
CLIENT_ID = 'SZPQQRN10VAR4YQTKNJLJLWFE5KU40KPSELLPFVV3IQCYTJM'
CLIENT_SECRET = 'QSGXPFTU2KHLWWKVNQ3MS11NWBMJRCMWMMKTQRN0TT5I4CVL'
VERSION = '20180605'

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
LIMIT = 500
mumbai_venues = getNearbyVenues(names=Neighborhoods['Neighborhood'],
                                   latitudes=Neighborhoods['Latitude'],
                                   longitudes=Neighborhoods['Longitude']
                                  )

Amboli
Chakala
D.N. Nagar
Four Bungalows
Lokhandwala
Marol
Model Town
Oshiwara
Poonam Nagar
Sahar
Saki Naka
Seven Bungalows
Versova
Mira Road
Bhayandar
Uttan
Kashimira
Bandra Kurla Complex
Gandhi Nagar
Kherwadi
Land's End
Pali Hill
Old Town
Bandra Reclamation
I.C. Colony
Shimpoli
Gorai
Kora Kendra
Vazira Naka
Babhai
Yogi Nagar
Magathane
Sukurwadi
Aarey Milk Colony
Motilal Nagar
Bangur Nagar
Gokuldham
Jayprakash Nagar
Jogeshwari West
Jogeshwari East
Charkop
Poisar
Hindustan Naka
Mahavir Nagar
Samta nagar
Damu nagar
Thakur complex
Thakur village
Akurli road
Hanuman nagar
Pali Naka
Khar Danda
Dindoshi
Sunder Nagar
Pathanwadi
Rani Sati Marg
Malvani
Kalina
Vakola
Naigaon
Irla
Hanuman Nagar
Pratap Nagar
Maharastra Nagar
JAmil NAgar
Utkarsh Nagar
samarth NAgar
Sahyadri NAgar
Sarvoday NAgar
Amrut Nagar
Garodia Nagar
Jagdusha Nagar
Pant Nagar
Nehru Nagar
Nahur
Chandivali
Hiranandani Gardens
Gandhi Nagar
Kannamwar Nagar
Vikhroli Park Site
Dayanand Saraswati Marg
P L Lokhande Marg
Thakkar Bappa C

### Now we distribute the venues into different categories and take the mean of thir occurance

In [12]:
mumbai_onehot = pd.get_dummies(mumbai_venues[['Venue Category']], prefix="", prefix_sep="")

mumbai_onehot['Neighborhood'] = mumbai_venues['Neighborhood'] 

fixed_columns = [mumbai_onehot.columns[-1]] + list(mumbai_onehot.columns[:-1])
mumbai_onehot = mumbai_onehot[fixed_columns]

In [13]:
mumbai_grouped = mumbai_onehot.groupby('Neighborhood').mean().reset_index()
mumbai_grouped

,Neighborhood,Yoga Studio,ATM,Afghan Restaurant,Airport,Airport Service,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Bengali Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Candy Store,Chaat Place,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,English Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Grocery Store,Gujarati Restaurant,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Halal Restaurant,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Malay Restaurant,Mamak Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Other Great Outdoors,Outlet Mall,Paper / Office Supplies Store,Park,Parsi Restaurant,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Print Shop,Pub,Public Art,Punjabi Restaurant,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Art,Supermarket,Swiss Restaurant,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Track,Train,Train Station,Tunnel,Udupi Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store
0,Aarey Milk Colony,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.

Now taking top 5 venues from all Neighborhoods

In [14]:
num_top_venues = 5

for hood in mumbai_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = mumbai_grouped[mumbai_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aarey Milk Colony----
                       venue  freq
0       Fast Food Restaurant   1.0
1  Indian Chinese Restaurant   0.0
2                      Motel   0.0
3            Motorcycle Shop   0.0
4              Movie Theater   0.0


----Agripada----
                venue  freq
0            Platform  0.25
1  Athletics & Sports  0.25
2         Coffee Shop  0.25
3              Bakery  0.25
4        Noodle House  0.00


----Akurli road----
          venue  freq
0         Plaza   0.2
1        Bakery   0.2
2   Coffee Shop   0.2
3  Soccer Field   0.2
4  Dessert Shop   0.2


----Altamount Road----
        venue  freq
0        Café  0.17
1  Restaurant  0.11
2     Theater  0.06
3     Brewery  0.06
4   Bookstore  0.06


----Amboli----
                venue  freq
0            Pharmacy  0.11
1                Park  0.11
2         Pizza Place  0.11
3      Sandwich Place  0.11
4  Chinese Restaurant  0.11


----Amrut Nagar----
               venue  freq
0  Indian Restaurant  0.20
1      Shopping M

                     venue  freq
0                     Park   0.5
1                      Bar   0.5
2              Yoga Studio   0.0
3  New American Restaurant   0.0
4                    Motel   0.0


----Jagdusha Nagar----
                           venue  freq
0              Indian Restaurant   0.4
1                  Hot Dog Joint   0.2
2                     Smoke Shop   0.2
3  Vegetarian / Vegan Restaurant   0.2
4        New American Restaurant   0.0


----Jayprakash Nagar----
                  venue  freq
0     Indian Restaurant  0.20
1  Fast Food Restaurant  0.13
2           Snack Place  0.13
3         Design Studio  0.07
4                 Hotel  0.07


----Jogeshwari East----
                     venue  freq
0       Mexican Restaurant  0.33
1            Moving Target  0.33
2     Gym / Fitness Center  0.33
3              Yoga Studio  0.00
4  New American Restaurant  0.00


----Jogeshwari West----
                  venue  freq
0             Multiplex  0.18
1                 Diner  0

                  venue  freq
0     Indian Restaurant  0.19
1  Fast Food Restaurant  0.15
2                  Café  0.07
3                   Pub  0.07
4           Pizza Place  0.04


----Samta nagar----
                 venue  freq
0          Bus Station  0.33
1    Indian Restaurant  0.17
2  Fried Chicken Joint  0.17
3                Diner  0.17
4   Chinese Restaurant  0.17


----Sarvoday NAgar----
                     venue  freq
0        Indian Restaurant  0.25
1              Snack Place  0.25
2                      Bar  0.25
3               Restaurant  0.25
4  New American Restaurant  0.00


----Seven Bungalows----
                venue  freq
0                 Pub  0.17
1                Café  0.13
2  Chinese Restaurant  0.10
3         Coffee Shop  0.07
4                 Bar  0.07


----Shimpoli----
                  venue  freq
0     Indian Restaurant  0.12
1           Men's Store  0.12
2                  Park  0.12
3  Fast Food Restaurant  0.12
4           Coffee Shop  0.12


----Si

Sorting the venues in ascending order

In [15]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Extracting top 10 venues for each Neighborhood

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = mumbai_grouped['Neighborhood']

for ind in np.arange(mumbai_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mumbai_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aarey Milk Colony,Fast Food Restaurant,Women's Store,Wine Bar,French Restaurant,Forest,Food Truck,Food Stand,Food Court,Food & Drink Shop,Food
1,Agripada,Platform,Coffee Shop,Athletics & Sports,Bakery,Women's Store,Fast Food Restaurant,Forest,Food Truck,Food Stand,Food Court
2,Akurli road,Dessert Shop,Plaza,Coffee Shop,Soccer Field,Bakery,Women's Store,Field,Forest,Food Truck,Food Stand
3,Altamount Road,Café,Restaurant,Brewery,Bakery,Park,Pizza Place,Concert Hall,Coffee Shop,Salon / Barbershop,Cafeteria
4,Amboli,Pharmacy,Indian Restaurant,Sandwich Place,Chinese Restaurant,Coffee Shop,Train Station,Gym,Pizza Place,Park,Dessert Shop


## Clustering Neighborhoods onto different cluster

In [32]:
kclusters = 4

mumbai_grouped_clustering = mumbai_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mumbai_grouped_clustering)

kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 0, 2, 0, 0, 2])

In [33]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

mumbai_merged = Neighborhoods

mumbai_merged = mumbai_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

mumbai_merged.head()

,Area,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Andheri,Amboli,19.127931,72.847735,2.0,Pharmacy,Indian Restaurant,Sandwich Place,Chinese Restaurant,Coffee Shop,Train Station,Gym,Pizza Place,Park,Dessert Shop
1,Andheri,Chakala,19.115287,72.861808,2.0,Hotel,Fast Food Restaurant,Indian Restaurant,Multiplex,Restaurant,Tea Room,Asian Restaurant,Seafood Restaurant,Chinese Restaurant,Pizza Place
2,Andheri,D.N. Nagar,19.128292,72.830193,0.0,Indian Restaurant,Women's Store,Restaurant,Liquor Store,Residential Building (Apartment / Condo),Cocktail Bar,Coffee Shop,Gym / Fitness Center,Pub,Lounge
3,Andheri,Four Bungalows,19.128794,72.825554,2.0,Pub,Women's Store,Asian Restaurant,Department Store,Café,Market,Fish Market,Residential Building (Apartment / Condo),Bar,Cocktail Bar
5,Andheri,Lokhandwala,19.143189,72.824081,0.0,Indian Restaurant,Café,Bakery,Pharmacy,Donut Shop,Fast Food Restaurant,Falafel Restaurant,Coffee Shop,Food Truck,Gym / Fitness Center


## Visualizing different clusters on map

In [34]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=9.5)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
mumbai_merged.dropna(axis = 0, inplace = True)

markers_colors = []
for lat, lon, poi, cluster in zip(mumbai_merged['Latitude'], mumbai_merged['Longitude'], mumbai_merged['Neighborhood'], mumbai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    c = int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[c-1],
        fill=True,
        fill_color=rainbow[c-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

## Looking at clusters seperately

### Cluster 1


In [20]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 0, mumbai_merged.columns[[1] + list(range(5, mumbai_merged.shape[1]))]].reset_index(0)

,index,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,D.N. Nagar,Indian Restaurant,Women's Store,Restaurant,Liquor Store,Residential Building (Apartment / Condo),Cocktail Bar,Coffee Shop,Gym / Fitness Center,Pub,Lounge
1,5,Lokhandwala,Indian Restaurant,Café,Bakery,Pharmacy,Donut Shop,Fast Food Restaurant,Falafel Restaurant,Coffee Shop,Food Truck,Gym / Fitness Center
2,6,Marol,Indian Restaurant,Hotel,Snack Place,Diner,Fast Food Restaurant,Forest,Food Truck,Food Stand,Food Court,Food & Drink Shop
3,9,Poonam Nagar,Yoga Studio,Restaurant,Indian Restaurant,Chinese Restaurant,Gym,Food Truck,Food Stand,Food Court,Food & Drink Shop,Food
4,11,Saki Naka,Indian Restaurant,Fast Food Restaurant,Café,Pub,Italian Restaurant,Sandwich Place,Brewery,Restaurant,Chinese Restaurant,Light Rail Station
5,17,Kashimira,Restaurant,Indian Restaurant,Motel,Women's Store,Food Truck,Food Stand,Food Court,Food & Drink Shop,Food,Flea Market
6,18,Bandra Kurla Complex,Indian Restaurant,Hotel,Café,Spa,Pizza Place,Sandwich Place,Gujarati Restaurant,Diner,Coffee Shop,Italian Restaurant
7,20,Gandhi Nagar,Indian Restaurant,Restaurant,Pizza Place,Department Store,Seafood Restaurant,Café,South American Restaurant,Chinese Restaurant,General Entertainment,Sports Club
8,21,Kherwadi,Indian Restaurant,Pizza Place,Tunnel,General Entertainment,Café,Diner,Gym / Fitness Center,Dhaba,Field,Food Stand
9,23,Pali Hill,Indian Restaurant,Asian Restaurant,Dessert Shop,Bar,Bakery,Seafood Restaurant,Café,Salad Place,Pub,Scenic Lookout


### Cluster 2

In [21]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 1, mumbai_merged.columns[[1] + list(range(5, mumbai_merged.shape[1]))]].reset_index(0)

,index,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,131,Navy Nagar,Garden,Women's Store,Fried Chicken Joint,Forest,Food Truck,Food Stand,Food Court,Food & Drink Shop,Food,Flea Market


### Cluster 3

In [22]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 2, mumbai_merged.columns[[1] + list(range(5, mumbai_merged.shape[1]))]].reset_index(0)

,index,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Amboli,Pharmacy,Indian Restaurant,Sandwich Place,Chinese Restaurant,Coffee Shop,Train Station,Gym,Pizza Place,Park,Dessert Shop
1,1,Chakala,Hotel,Fast Food Restaurant,Indian Restaurant,Multiplex,Restaurant,Tea Room,Asian Restaurant,Seafood Restaurant,Chinese Restaurant,Pizza Place
2,3,Four Bungalows,Pub,Women's Store,Asian Restaurant,Department Store,Café,Market,Fish Market,Residential Building (Apartment / Condo),Bar,Cocktail Bar
3,7,Model Town,Chinese Restaurant,Fast Food Restaurant,Gym / Fitness Center,Park,Wine Bar,Ice Cream Shop,Shoe Store,Indian Restaurant,Department Store,Sandwich Place
4,8,Oshiwara,Café,Indian Restaurant,Clothing Store,Bar,Chinese Restaurant,Coffee Shop,Shopping Mall,Sporting Goods Shop,Bengali Restaurant,Cocktail Bar
5,10,Sahar,Airport,Women's Store,Fast Food Restaurant,French Restaurant,Forest,Food Truck,Food Stand,Food Court,Food & Drink Shop,Food
6,12,Seven Bungalows,Pub,Café,Chinese Restaurant,Coffee Shop,Bar,Juice Bar,Bistro,South Indian Restaurant,Japanese Restaurant,Italian Restaurant
7,13,Versova,Pub,Café,Chinese Restaurant,Bar,Coffee Shop,Juice Bar,South Indian Restaurant,Bistro,Italian Restaurant,Asian Restaurant
8,14,Mira Road,Clothing Store,Pub,Bar,Café,Chinese Restaurant,Coffee Shop,Shopping Mall,Women's Store,Garden,Sandwich Place
9,15,Bhayandar,Resort,Playground,Dessert Shop,Moving Target,Lake,Falafel Restaurant,Food Stand,Food Court,Food & Drink Shop,Food


### Cluster 4

In [23]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 3, mumbai_merged.columns[[1] + list(range(5, mumbai_merged.shape[1]))]].reset_index(0)

,index,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,71,Irla,Park,Bar,Women's Store,Fast Food Restaurant,Forest,Food Truck,Food Stand,Food Court,Food & Drink Shop,Food


## Observations and Conclusion

Looking at the different clusters and its venues we can say that 
1. Neighborhoods in cluster 1 and 2 seems to be residential and food trucks can be used for testing before opening a poper restaurant or pop up shops.

2. Neighborhoods in cluster 3 i.e. Arey Milk colony can benifit from a food truck

3. Neighborhoods in cluster 4 appear to be a mix of buisness and residential are, hence restaurants can be setup in these areas